## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-12-10-49-00 +0000,wsj,Russia’s Central Bank Cuts Key Rate for Third ...,https://www.wsj.com/articles/russias-central-b...
1,2025-09-12-10-47-22 +0000,nypost,Meghan Markle’s half sister Samantha suffers s...,https://nypost.com/2025/09/12/us-news/meghan-m...
2,2025-09-12-10-41-04 +0000,bbc,Lords urged to scrutinise but not block assist...,https://www.bbc.com/news/articles/cly0d1vg0e2o...
3,2025-09-12-10-40-30 +0000,nyt,Lawyers for Venezuelans Ask Court to Press D.H...,https://www.nytimes.com/2025/09/11/us/lawyers-...
4,2025-09-12-10-30-00 +0000,nypost,Garden City high school’s 54-game win streak o...,https://nypost.com/2025/09/12/sports/garden-ci...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2322/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,kirk,63
65,charlie,53
90,trump,35
68,shooting,18
521,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
33,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,213
160,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,199
343,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free-sp...,https://www.latimes.com/california/story/2025-...,180
77,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...,178
251,2025-09-11-18-02-59 +0000,nyt,Trump Says He Will Award Charlie Kirk the Pres...,https://www.nytimes.com/2025/09/11/us/charlie-...,176


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
33,213,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
160,83,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
77,62,2025-09-12-06-29-26 +0000,wapo,Officials release new video of person fleeing ...,https://www.washingtonpost.com/nation/2025/09/...
54,47,2025-09-12-08-38-23 +0000,nypost,Prince Harry makes surprise visit to Kyiv as h...,https://nypost.com/2025/09/12/world-news/princ...
211,46,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
128,41,2025-09-12-01-10-28 +0000,bbc,Man accused of Trump assassination attempt cut...,https://www.bbc.com/news/articles/c8d793z4mzgo...
67,40,2025-09-12-08-00-00 +0000,cbc,"Israel's attack on Qatar was an escalation, bu...",https://www.cbc.ca/news/world/israel-qatar-inv...
57,39,2025-09-12-08-21-06 +0000,nypost,"Josh Kraft, son of billionaire Patriots owner,...",https://nypost.com/2025/09/12/us-news/robert-k...
5,37,2025-09-12-10-18-00 +0000,wsj,Paramount Skydance Prepares Ellison-Backed Bid...,https://www.wsj.com/business/media/paramount-s...
0,35,2025-09-12-10-49-00 +0000,wsj,Russia’s Central Bank Cuts Key Rate for Third ...,https://www.wsj.com/articles/russias-central-b...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
